**What Is Amortization?**

Amortization is an accounting technique used to periodically lower the book value of a loan or an intangible asset over a set period of time. Concerning a loan, amortization focuses on spreading out loan payments over time. When applied to an asset, amortization is similar to depreciation.

How to Calculate Amortization of Loans
The formula to calculate the monthly principal due on an amortized loan is as follows:

\begin{aligned}&\text{Principal Payment} = \text{TMP} - \Big ( \text{OLB} \times \frac { \text{Interest Rate} }{ \text{12 Months} } \Big ) \\&\textbf{where:} \\&\text{TMP} = \text{Total monthly payment} \\&\text{OLB} = \text{Outstanding loan balance} \\\end{aligned} 
​
  
Principal Payment=TMP−(OLB× 
12 Months
Interest Rate
​
 )
where:
TMP=Total monthly payment
OLB=Outstanding loan balance
​


In [75]:
# Install numpy-financial
#The numpy-financial Python package is a collection of elementary financial functions
!pip install numpy-financial

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [76]:
# import the appropriate libraries

import pandas as pd
import datetime as dt
import numpy_financial as npf
import numpy as np
from IPython.display import display, HTML
import plotly.express as px
from tabulate import tabulate

In [77]:
# To round the number within the dataframe
# fmt =pd.options.display.float_format = '{:6.2f}'.format

In [78]:
# Get the user input
principal = int(input("Enter the loan amount: ")) #present value (pv)

loan_term = int(input("Enter the loan term (in years): "))

int_rate = float(input("Enter the interest rate: "))

Enter the loan amount: 20000
Enter the loan term (in years): 4
Enter the interest rate: 7.99


In [79]:
n_period = loan_term * 12 # Number of compounding periods
interest_rate = int_rate /1200 # Rate of interest (per period)

# Calculate monthly payment (pmt)   
#Formula ----> pmt(rate, nper, pv, fv=0, when='end')
monthly_payment = npf.pmt(interest_rate, n_period, -principal)
# monthly_payment

# The total payment is made up of payment against principal plus interest.
# Monthly interest ---> #ipmt(rate, per, nper, pv, fv=0, when='end')
#The 'per' variable represents the periods of the loan. Remember that financial equations start the period count at 1!
period = np.arange(n_period) + 1 
monthly_interest_paid = npf.ipmt(interest_rate, period,n_period, -principal)
monthly_interest_paid

#payment against loan principal.. monthly_payment - monthly_interest_paid
new_principal = npf.ppmt(interest_rate, period, n_period, -principal)
# print(f"PPMT{new_principal}")

#Each element of the sum of the 'ipmt' and 'ppmt' arrays should equal 'pmt'.``pmt = ppmt + ipmt``
np.allclose(monthly_interest_paid + new_principal, monthly_payment) #Returns True if the two arrays are equal within the given tolerance;


True

In [80]:
# Calculate the period lenght
final_date = (n_period)
drange = pd.date_range(start =dt.datetime.today(), periods= final_date,freq='M')
drange =pd.to_datetime(drange)
drange = drange.strftime("%b %Y")
# drange


In [81]:
#calculate the sum of interest paid
sum_interest = round(np.sum(monthly_interest_paid),2)
print(f"${sum_interest}")

$3431.9


In [82]:
#prepare the dataframe                         
data = pd.DataFrame({'Date': drange,
                     'Principal +': new_principal,
                     'Interest':monthly_interest_paid,
                     '= Payment':monthly_payment
                     })
data['Total Interest Paid'] = data['Interest'].cumsum() 
data['Balance'] = principal - data['Principal +'].cumsum()

#Make of a copy of the original data to display the amortization in $ format
amortization = data.copy()

# adds the $ through style formatting
formatters={'Principal +':lambda x: "$ {:,.2f}".format(x), 
            'Interest': lambda x: "$ {:,.2f}".format(x), 
            '= Payment': lambda x: "$ {:,.2f}".format(x),
            'Total Interest Paid': lambda x: "$ {:,.2f}".format(x),
            'Balance': lambda x: "$ {:,.2f}".format(x)}

# amortization =display(amortization.style.format(formatters))

In [83]:
#Make another copy of the original data to work with the rest of the data (optional)
results=data.copy()
results.head().style.format(formatters)

,Date,Principal +,Interest,= Payment,Total Interest Paid,Balance
0,Jul 2022,$ 355.00,$ 133.17,$ 488.16,$ 133.17,"$ 19,645.00"
1,Aug 2022,$ 357.36,$ 130.80,$ 488.16,$ 263.97,"$ 19,287.64"
2,Sep 2022,$ 359.74,$ 128.42,$ 488.16,$ 392.39,"$ 18,927.90"
3,Oct 2022,$ 362.14,$ 126.03,$ 488.16,$ 518.42,"$ 18,565.76"
4,Nov 2022,$ 364.55,$ 123.62,$ 488.16,$ 642.04,"$ 18,201.22"


In [84]:
# # function definition
def highlight_cols(x):
      
    # copy df to new - original data is not changed
    results = x.copy()
    
    # select all values to green color
    results.loc[:, :] = 'background-color: lavender'
    
    # overwrite values grey color
    results[['Principal +']] = 'background-color:#008F8D'
    results[['Interest']] = 'background-color:#852668' 
    results[['= Payment']] = 'background-color:#D84C2C'
    # return color df
    return results


display(results.style.format(formatters).apply(highlight_cols, axis = None))


,Date,Principal +,Interest,= Payment,Total Interest Paid,Balance
0,Jul 2022,$ 355.00,$ 133.17,$ 488.16,$ 133.17,"$ 19,645.00"
1,Aug 2022,$ 357.36,$ 130.80,$ 488.16,$ 263.97,"$ 19,287.64"
2,Sep 2022,$ 359.74,$ 128.42,$ 488.16,$ 392.39,"$ 18,927.90"
3,Oct 2022,$ 362.14,$ 126.03,$ 488.16,$ 518.42,"$ 18,565.76"
4,Nov 2022,$ 364.55,$ 123.62,$ 488.16,$ 642.04,"$ 18,201.22"
5,Dec 2022,$ 366.97,$ 121.19,$ 488.16,$ 763.23,"$ 17,834.24"
6,Jan 2023,$ 369.42,$ 118.75,$ 488.16,$ 881.97,"$ 17,464.82"
7,Feb 2023,$ 371.88,$ 116.29,$ 488.16,$ 998.26,"$ 17,092.94"
8,Mar 2023,$ 374.35,$ 113.81,$ 488.16,"$ 1,112.07","$ 16,718.59"
9,Apr 2023,$ 376.85,$ 111.32,$ 488.16,"$ 1,223.39","$ 16,341.74"


In [85]:
drange =pd.to_datetime(drange)
date_in_year =drange.strftime("%Y")
results['Years'] = date_in_year
results['Years'] 
a =results[['Years','Principal +','Interest','Balance']]
a.groupby(['Years']).sum()
print(tabulate(a.groupby(['Years']).sum(), headers=a, tablefmt='fancy_grid'))

╒═════════╤═══════════════╤════════════╤═══════════╕
│   Years │   Principal + │   Interest │   Balance │
╞═════════╪═══════════════╪════════════╪═══════════╡
│    2022 │       2165.76 │   763.228  │  112462   │
├─────────┼───────────────┼────────────┼───────────┤
│    2023 │       4599.02 │  1258.96   │  184481   │
├─────────┼───────────────┼────────────┼───────────┤
│    2024 │       4980.24 │   877.736  │  126845   │
├─────────┼───────────────┼────────────┼───────────┤
│    2025 │       5393.06 │   464.914  │   64431.4 │
├─────────┼───────────────┼────────────┼───────────┤
│    2026 │       2861.92 │    67.0636 │    7210.2 │
╘═════════╧═══════════════╧════════════╧═══════════╛


In [86]:
import plotly.graph_objects as go

colors = ['#008F8D',] *3
colors[1] = '#852668'
colors[2] = '#D84C2C'
x=['Principal', 'Interest', 'Total Paid']

y=round(results[['Principal +','Interest','= Payment']].sum(),2)
fig = go.Figure(data=[go.Bar(
     x=x, y=y,text=y, textposition='auto',
    marker_color=colors, # marker color can be a single color value or an iterable
    width=[0.5, 0.5, 1.0]
)])
# fig.update_layout(title_text='Least Used Feature')
fig.show()

In [87]:
import plotly.graph_objects as go

# years = [1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003,
#          2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012]
fig = go.Figure()
# fig.add_trace(go.Bar(x=results['Years'] ,
#                 y=results['= Payment'],
#                 name='Principal',
#                 marker_color='#D84C2C'
#                 ))
fig.add_trace(go.Bar(x=results['Years'] ,
                y=results['Total Interest Paid'],
                name='Compound Interest',
                marker_color='#852668'
                ))

fig.update_layout(
    title='Payment VS Interest',
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='Compound Interest',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15, # gap between bars of adjacent location coordinates.
    bargroupgap=0.1 # gap between bars of the same location coordinate.
)
fig.show()